In [1]:
%matplotlib inline

import numpy as np
import h5py

import torch
from torch.nn import functional as F
from torch.autograd import Variable

##############################
from models import deeplab_xception
from models import tiramisu
from models import unet
from datasets import datasets
from utils import manager as mgr
from utils import img_utils
##############################

# set device
#device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
#exp_name = 'unet_df_030_001'
#WEIGHTS_PATH = '/home/philipp/Data/weights/'+exp_name+'/'
WEIGHTS_PATH = '/home/philipp/Code/python/edin_prediction/weights/'
device = "cuda:0"
#device = "cpu"
print(device)

architecture = 'deeplab'#'deeplab'#'densenet'
nr_channels = 7
nr_classes = 4

cuda:0


In [2]:
## load model with weights

In [3]:
if architecture == 'unet':
    model = unet.UNET(in_channels=nr_channels, out_channels=nr_classes)
    weights_name = 'unet_dice_weights-19-0.217-0.821.pth'
    
elif architecture == 'densenet':
    model = tiramisu.FCDenseNet57(in_channels=nr_channels, n_classes=nr_classes)
    weights_name = 'densenet_dice_weights-3-0.186-0.808.pth'
    
elif architecture == 'deeplab':
    model = deeplab_xception.DeepLabv3_plus(nInputChannels=nr_channels, n_classes=nr_classes, os=16, pretrained=False)
    weights_name = 'deeplab_dice_weights-21-0.192-0.818.pth'
    
elif architecture == 'deeplab_ce':
    model = deeplab_xception.DeepLabv3_plus(nInputChannels=nr_channels, n_classes=nr_classes, os=16, pretrained=False)
    weights_name = 'deeplab_ce_weights-4-0.453-0.814.pth'
    
else:
    print('provide architecture')
    
try:
    mgr.load_weights(model, WEIGHTS_PATH+weights_name)
    print("weights loaded")
except:
    model.apply(mgr.weights_init)
    print("no weights found")
    
model.to(device)

Constructing DeepLabv3+ model...
Number of classes: 4
Output stride: 16
Number of Input Channels: 7
loading weights '/home/philipp/Code/python/edin_prediction/weights/deeplab_dice_weights-21-0.192-0.818.pth'
loaded weights (lastEpoch 20, loss 0.19151103496551514, error 0.8182618618011475)
weights loaded


DeepLabv3_plus(
  (xception_features): Xception(
    (conv1): Conv2d(7, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d_same(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running

In [4]:
## load dataset

In [5]:
class ForestDataset(torch.utils.data.Dataset):

    '''Characterizes a dataset for PyTorch'''

    def __init__(self, path):
        '''Initialization'''
        # open dataset
        self.dset = h5py.File(path, 'r')
        self.ortho = self.dset['ortho']
        self.dsm = self.dset['dsm']
        self.dtm = self.dset['dtm']
        self.slope = self.dset['slope']

        # set number of samples
        self.dataset_size = 10679

        ## TODO:
        # make means and stds load from hdf5
        self.means_tams = np.array([56.12055784563426, 62.130400134006976, 53.03228547781888, 119.50916281232037], dtype='float32')
        self.stds_tams = np.array([30.37628560708646, 30.152693706272483, 23.13718651792004, 49.301477498205074], dtype='float32')

        self.means_dsm = np.array([13.45]).astype(np.float32)
        self.stds_dsm = np.array([10.386633252098674]).astype(np.float32)

        self.means_dtm = np.array([1446.0]).astype(np.float32)
        self.stds_dtm = np.array([271.05322202384195]).astype(np.float32)

        self.means_slope = np.array([22.39]).astype(np.float32)
        self.stds_slope = np.array([11.69830556896441]).astype(np.float32)


    def __len__(self):
        '''Denotes the total number of samples'''
        return self.dataset_size


    def __getitem__(self, index):
        '''Generates one sample of data'''

        # depending on data change mean and std
        means = self.means_tams
        stds = self.stds_tams

        # Load data and get label
        X_ortho = (torch.tensor(self.ortho[index], \
            dtype=torch.float32).permute(2, 0, 1) - \
            means[:, np.newaxis, np.newaxis]) / stds[:, np.newaxis, np.newaxis]
        X_dsm = (torch.tensor(self.dsm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dsm) / self.stds_dsm
        X_dtm = (torch.tensor(self.dtm[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_dtm) / self.stds_dtm
        X_slope = (torch.tensor(self.slope[index], \
            dtype=torch.float32).permute(2, 0, 1) - self.means_slope) / self.stds_slope

        X = torch.cat((X_ortho, X_dsm, X_dtm, X_slope),0)

        return X #torch.from_numpy(y).permute(2, 0, 1)


    def close(self):
        ''' closes the hdf5 file'''
        self.dset.close()


    def show_item(self, index):
        '''shows the data'''
        #plt.imshow(np.array(self.ground_truth[index]))

        fig = plt.figure(figsize=(20,20))

        dic_data = {'RGB' : [np.array(self.ortho[index][:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'CIR' : [np.array(np.roll(self.ortho[index], 1, axis=2)[:,:,:3]), [0.1, 0.3, 0.5, 0.7]], \
        'DSM' : [np.array(self.dsm[index].astype('f')), [10, 20, 30]], \
        'DTM' : [np.array(self.dtm[index].astype('f')), [10, 20, 30]], \
        'Slope' : [np.array(self.slope[index].astype('f')), [10, 20, 30]]}

        for i, key in enumerate(dic_data):
            ax = fig.add_subplot(2, 3, i+1)
            imgplot = plt.imshow(dic_data[key][0])
            ax.set_title(key)
            plt.colorbar(ticks=dic_data[key][1], orientation='horizontal')
            plt.axis('off')

In [6]:
## loading the dataset
#path_dataset = "/home/philipp/Data/dataset_256_df_2.h5"
path_dataset = "/media/philipp/DATA/dataset/dataset_512_df_prediction.h5"
#path_dataset = "/media/philipp/DATA/dataset/dataset_256_df_2.h5"

# open dataset
dataset = ForestDataset(path_dataset)

In [7]:
def quatering(dataset, idx_start, idx_end):
    ## prepare input data
    # cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256
    n = idx_end - idx_start
    store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

    for j,i in enumerate(range(idx_start, idx_end)):
        store[j*4] = dataset[i][:,:256,:256]
        store[j*4+1] = dataset[i][:,256:,:256]
        store[j*4+2] = dataset[i][:,:256,256:]
        store[j*4+3] = dataset[i][:,256:,256:] 
    return store

In [8]:
def predict(store):
    # get predictions
    with torch.no_grad():
        output = model(store)
    pred_256 = mgr.get_predictions(output).numpy()
    return pred_256

In [9]:
def merge(pred_256, idx_start, idx_end):
    ## prepare output data
    # merge into 4 quaters = 4 x 256x256 -> 1 x 512x512
    n = idx_end - idx_start
    pred_512 = np.zeros((n,512,512), dtype=np.int8)
    
    for i in range(n):
        j = i*4
        pred_512[i,:256,:256] = pred_256[j]
        pred_512[i,256:,:256] = pred_256[j+1]
        pred_512[i,:256,256:] = pred_256[j+2]
        pred_512[i,256:,256:] = pred_256[j+3]
    return pred_512    

In [10]:
def stack_data(dataset, idx_start, idx_end):
    dl = []
    for i in range(idx_start, idx_end):
        dl.append(dataset[i])
    return torch.stack(dl,0)

In [14]:
start = 0
end = 10660
batch = 40

p = np.zeros((end-start,512,512), dtype=np.int8)
print(p.shape)

counter = start

for i in range((end-start) // batch):
    
    print(counter, counter+batch)
    
    #store = quatering(dataset=dataset, idx_start=counter, idx_end=counter+batch)
    store = stack_data(dataset=dataset, idx_start=counter, idx_end=counter+batch)
    store = store.to(device)
    pred_512 = predict(store)
    #pred_512 = pred_512.to('cpu')
    #pred_512 = merge(pred_256, counter, counter+batch)
    
    p[i*batch:(i+1)*batch] = pred_512
    
    counter += batch
    


(10660, 512, 512)
0 40
40 80
80 120
120 160
160 200
200 240
240 280
280 320
320 360
360 400
400 440
440 480
480 520
520 560
560 600
600 640
640 680
680 720
720 760
760 800
800 840
840 880
880 920
920 960
960 1000
1000 1040
1040 1080
1080 1120
1120 1160
1160 1200
1200 1240
1240 1280
1280 1320
1320 1360
1360 1400
1400 1440
1440 1480
1480 1520
1520 1560
1560 1600
1600 1640
1640 1680
1680 1720
1720 1760
1760 1800
1800 1840
1840 1880
1880 1920
1920 1960
1960 2000
2000 2040
2040 2080
2080 2120
2120 2160
2160 2200
2200 2240
2240 2280
2280 2320
2320 2360
2360 2400
2400 2440
2440 2480
2480 2520
2520 2560
2560 2600
2600 2640
2640 2680
2680 2720
2720 2760
2760 2800
2800 2840
2840 2880
2880 2920
2920 2960
2960 3000
3000 3040
3040 3080
3080 3120
3120 3160
3160 3200
3200 3240
3240 3280
3280 3320
3320 3360
3360 3400
3400 3440
3440 3480
3480 3520
3520 3560
3560 3600
3600 3640
3640 3680
3680 3720
3720 3760
3760 3800
3800 3840
3840 3880
3880 3920
3920 3960
3960 4000
4000 4040
4040 4080
4080 4120
4120 41

In [15]:
p.shape

(10660, 512, 512)

In [17]:
store = stack_data(dataset=dataset, idx_start=10640, idx_end=10660)
store = store.to(device)
pred_512 = predict(store)
p[10640:10660] = pred_512

In [18]:
np.save('pred_deeplab.npy', p)

In [ ]:
##################################################################

In [1]:
## prepare input data
# cut into 4 quaters = 1 x 7x512x512 -> 4 x 7x256x256

start = 0
end = 40

n = end - start
store = torch.zeros((n*4,7,256,256),dtype=torch.float32)

for i,j in enumerate(range(0,n*4,4)):
    store[j] = dataset[i][:,:256,:256]
    store[j+1] = dataset[i][:,256:,:256]
    store[j+2] = dataset[i][:,:256,256:]
    store[j+3] = dataset[i][:,256:,256:]

NameError: name 'torch' is not defined

In [10]:
# get predictions
with torch.no_grad():
    output = model(store)
p = mgr.get_predictions(output).numpy()

In [11]:
p.shape

(160, 256, 256)

In [12]:
## prepare output data
# merge into 4 quaters = 4 x 7x256x256 -> 1 x 7x512x512

pred = np.zeros((n,512,512), dtype=np.int8)

for i,j in enumerate(range(0,n*4,4)):
    pred[i,:256,:256] = p[j]
    pred[i,256:,:256] = p[j+1]
    pred[i,:256,256:] = p[j+2]
    pred[i,256:,256:] = p[j+3]

In [13]:
pred.shape

(40, 512, 512)

In [12]:
mean = 119.50916281232037
std = 49.301477498205074

i = 0
l = 10000
nirs = []

for i in range(20):
    # grab data
    x = torch.stack([dataset[i][3], \
                    dataset[i+1*l][3], \
                    dataset[i+2*l][3], \
                    dataset[i+3*l][3]])
    
    # reconstruct tile
    merged_512 = np.ones((512, 512))
    # paste predicted data into array
    merged_512[:256,:256] = dataset[i][3]
    merged_512[256:,:256] = dataset[i+1*l][3]
    merged_512[:256,256:] = dataset[i+2*l][3]
    merged_512[256:,256:] = dataset[i+3*l][3]
    
    merged_512 = std * merged_512 + mean
    
    nirs.append(merged_512)

In [13]:
nirs = np.array(nirs)

In [15]:
np.save('nir.npy', nirs)

In [16]:
np.save('pred.npy', p)

In [1]:
## convert to geotiff

In [1]:
import os
import numpy as np

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [2]:
def array2raster(newRasterfn, dataset, array, dtype):
    """
    save GTiff file from numpy.array
    input:
        newRasterfn: save file name
        dataset : original tif file
        array : numpy.array
        dtype: Byte or Float32.
    """
    cols = array.shape[1]
    rows = array.shape[0]
    originX, pixelWidth, b, originY, d, pixelHeight = dataset.GetGeoTransform() 

    driver = gdal.GetDriverByName('GTiff')

    # set data type to save.
    GDT_dtype = gdal.GDT_Unknown
    if dtype == "Byte": 
        GDT_dtype = gdal.GDT_Byte
    elif dtype == "Float32":
        GDT_dtype = gdal.GDT_Float32
    else:
        print("Not supported data type.")

    # set number of band.
    if array.ndim == 2:
        band_num = 1
    else:
        band_num = array.shape[2]

    outRaster = driver.Create(newRasterfn, cols, rows, band_num, GDT_dtype)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))

    # Loop over all bands.
    for b in range(band_num):
        outband = outRaster.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        if band_num == 1:
            outband.WriteArray(array)
        else:
            outband.WriteArray(array[:,:,b])

    # setteing srs from input tif file.
    prj=dataset.GetProjection()
    outRasterSRS = osr.SpatialReference(wkt=prj)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

In [3]:
pred = np.load('pred_deeplab.npy')
#pred = np.load('pred_deeplab.npy')

In [4]:
pred.shape

(10660, 512, 512)

In [5]:
#########################
### extend 512 to 514 ###
#########################

# create array of 514
pred_p = np.zeros((pred.shape[0], 514, 514),dtype=np.int8)
# set inner part to real data
pred_p[:,1:513,1:513] = pred

# pad rows (copy nearest row)
pred_p[:,0,1:513] = pred[:,0,:]
pred_p[:,-1,1:513] = pred[:,-1,:]
# pad columns (copy nearest row)
pred_p[:,1:513,0] = pred[:,:,0]
pred_p[:,1:513,-1] = pred[:,:,-1]

# fill corners
pred_p[:,0,0] = pred[:,0,0]
pred_p[:,0,-1] = pred[:,0,-1]
pred_p[:,-1,0] = pred[:,-1,0]
pred_p[:,-1,-1] = pred[:,-1,-1]

In [6]:
def find_files(dir_img, sort=False):
    """
    find paths for provided data types
    inputs:
        dir_img (str) : directory path
    return:
        paths (dictionary) : dictionary containing file paths for each of the data types
    """

    paths = []
    # loop over all files found in directory and retrive indices
    for file in os.listdir("{}".format(dir_img)):
        if file[-4:] == ".tif":
            paths.append(dir_img + file)

    if sort:
        paths = sorted(paths)
    
    return paths

In [7]:
path_input = find_files('/media/philipp/DATA/2018_tamsweg/ortho/', sort=True)

In [8]:
path_input[0:3]

['/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121889.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_121890.tif',
 '/media/philipp/DATA/2018_tamsweg/ortho/tile_ortho_122277.tif']

In [9]:
len(path_input)

10679

In [10]:
pred.shape[0]

10660

In [11]:
for i, path_in in enumerate(path_input[:pred.shape[0]]):
    print(path_in[path_in.rfind('_'):])

_121889.tif
_121890.tif
_122277.tif
_122278.tif
_122279.tif
_122280.tif
_122647.tif
_122648.tif
_122649.tif
_122650.tif
_122651.tif
_122652.tif
_122653.tif
_123009.tif
_123010.tif
_123011.tif
_123012.tif
_123013.tif
_123014.tif
_123015.tif
_123016.tif
_123017.tif
_123349.tif
_123350.tif
_123351.tif
_123352.tif
_123353.tif
_123354.tif
_123355.tif
_123356.tif
_123357.tif
_123658.tif
_123659.tif
_123660.tif
_123661.tif
_123662.tif
_123663.tif
_123664.tif
_123949.tif
_123950.tif
_123951.tif
_123952.tif
_124241.tif
_124242.tif
_124243.tif
_124515.tif
_124516.tif
_124517.tif
_124518.tif
_124519.tif
_124520.tif
_124521.tif
_124796.tif
_124797.tif
_124798.tif
_124799.tif
_124800.tif
_124801.tif
_124802.tif
_124803.tif
_124804.tif
_124805.tif
_124806.tif
_124807.tif
_125073.tif
_125074.tif
_125075.tif
_125076.tif
_125077.tif
_125078.tif
_125079.tif
_125080.tif
_125081.tif
_125082.tif
_125368.tif
_125369.tif
_125370.tif
_125371.tif
_125372.tif
_125373.tif
_125374.tif
_125375.tif
_125671.tif
_125

_180693.tif
_180694.tif
_180695.tif
_180828.tif
_180829.tif
_180830.tif
_180831.tif
_180832.tif
_180833.tif
_180834.tif
_180835.tif
_180836.tif
_180837.tif
_180838.tif
_180962.tif
_180963.tif
_180964.tif
_180965.tif
_180966.tif
_180967.tif
_180968.tif
_180969.tif
_181092.tif
_181093.tif
_181094.tif
_181095.tif
_181096.tif
_181097.tif
_181098.tif
_181099.tif
_181217.tif
_181218.tif
_181219.tif
_181220.tif
_181221.tif
_181222.tif
_181223.tif
_181224.tif
_181225.tif
_181341.tif
_181342.tif
_181343.tif
_181344.tif
_181345.tif
_181346.tif
_181347.tif
_181348.tif
_181349.tif
_181350.tif
_181466.tif
_181467.tif
_181468.tif
_181469.tif
_181470.tif
_181471.tif
_181472.tif
_181473.tif
_181474.tif
_181590.tif
_181591.tif
_181592.tif
_181593.tif
_181594.tif
_181595.tif
_181596.tif
_181597.tif
_181598.tif
_181599.tif
_181711.tif
_181712.tif
_181713.tif
_181714.tif
_181715.tif
_181716.tif
_181717.tif
_181718.tif
_181821.tif
_181822.tif
_181823.tif
_181824.tif
_181825.tif
_181826.tif
_181827.tif
_181

_207897.tif
_207898.tif
_207899.tif
_207900.tif
_207901.tif
_207902.tif
_207903.tif
_207904.tif
_207905.tif
_207906.tif
_207907.tif
_207908.tif
_208022.tif
_208023.tif
_208024.tif
_208025.tif
_208026.tif
_208027.tif
_208028.tif
_208029.tif
_208030.tif
_208031.tif
_208032.tif
_208033.tif
_208034.tif
_208035.tif
_208036.tif
_208037.tif
_208038.tif
_208143.tif
_208144.tif
_208145.tif
_208146.tif
_208147.tif
_208148.tif
_208149.tif
_208150.tif
_208151.tif
_208152.tif
_208153.tif
_208154.tif
_208155.tif
_208156.tif
_208157.tif
_208158.tif
_208159.tif
_208160.tif
_208161.tif
_208162.tif
_208163.tif
_208164.tif
_208253.tif
_208254.tif
_208255.tif
_208256.tif
_208257.tif
_208258.tif
_208259.tif
_208260.tif
_208261.tif
_208262.tif
_208263.tif
_208264.tif
_208265.tif
_208266.tif
_208267.tif
_208268.tif
_208269.tif
_208270.tif
_208271.tif
_208272.tif
_208273.tif
_208274.tif
_208348.tif
_208349.tif
_208350.tif
_208351.tif
_208352.tif
_208353.tif
_208354.tif
_208355.tif
_208356.tif
_208357.tif
_208

In [12]:
# convert numpy array to geotiff files
for i, path_in in enumerate(path_input[:pred.shape[0]]):
    path_out = 'foto/pred{}'.format(path_in[path_in.rfind('_'):])
    dataset = gdal.Open(path_in, gdal.GA_ReadOnly)
    array2raster(newRasterfn=path_out, dataset=dataset, array=pred_p[i], dtype='Byte')

In [13]:
# mosaik geotiff files

def mosaik_raster(input_files, output_file):
    
    # set path to text file with all tif file paths to be merged
    merge_list_path =  '/home/philipp/Code/python/edin_prediction/dtm_merge_list.txt'
    # create txt file with all tif files needed
    with open(merge_list_path, 'w+') as f:
        for name in input_files:
            f.write(name + '\n')

    # create string containing bash command
    cmd = "gdal_merge.py -ot Byte -of GTiff -o {} --optfile {}".format(output_file, merge_list_path)

    # execute bash command
    os.system(cmd)

    print("DTM file created - files merged")

In [14]:
input_files = find_files('/home/philipp/Code/python/edin_prediction/foto/', sort=True)

In [15]:
#output_file = '/home/philipp/Code/python/edin_prediction/pred/pred.tif'
output_file = '/media/philipp/DATA/pred_deeplab.tif'
mosaik_raster(input_files, output_file)

DTM file created - files merged


In [16]:
vector_file = '/home/philipp/Code/python/edin_prediction/pred/pred_deeplab_512.shp'
# create vector (polygons) out of raster
cmd = 'gdal_polygonize.py -8 {} -b 1 -f "ESRI Shapefile" {} output DF'.format(output_file, vector_file)
# execute bash command
os.system(cmd)

0

In [26]:
42/160*10679

2803.2375

In [19]:
output_file

'/media/philipp/DATA/pred.tif'

In [1]:
################################################
# confusion matrix
################################################

import os
import numpy as np

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

In [3]:
def tif2array(file_path, dtype=np.uint8):
        """
        read GeoTiff and convert to numpy.ndarray.
        inputs:
            file_path (str) : file path of the input GeoTiff file
        return:
            image(np.array) : image for each bands
            dataset : for gdal's data drive
        """
        dataset = gdal.Open(file_path, gdal.GA_ReadOnly)

        if dataset is None:
            return None

        # Allocate our array using the first band's datatype
        image_datatype = dataset.GetRasterBand(1).DataType
        image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                         dtype=dtype)

        # Loop over all bands in dataset
        for b in range(dataset.RasterCount):
            # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
            band = dataset.GetRasterBand(b + 1)
            # Read in the band's data into the third dimension of our array
            image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)

        #image = image[2:-2,2:-2,:]

        return image

In [4]:
input_files = find_files('/home/philipp/Code/python/edin_prediction/foto/', sort=True)

In [5]:
dir_gt = "/media/philipp/DATA/2018_tamsweg/ground_truth_2/tile_ground_truth"
#y = tif2array("{}{}".format(dir_gt, input_files[0][input_files[0].rfind('_'):]))[1:-2, 1:-2]
#y_hat = tif2array(input_files[0])

In [6]:
d_y = []
#d_y_hat = []

for f in input_files:
    d_y.append(tif2array("{}{}".format(dir_gt, f[f.rfind('_'):]))[1:-2, 1:-2])
    #d_y_hat.append(tif2array(f))
    print(f)

/home/philipp/Code/python/edin_prediction/foto/pred_121889.tif
/home/philipp/Code/python/edin_prediction/foto/pred_121890.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122277.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122278.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122279.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122280.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122647.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122648.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122649.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122650.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122651.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122652.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122653.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123009.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123010.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123

/home/philipp/Code/python/edin_prediction/foto/pred_130427.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130428.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130644.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130645.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130646.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130647.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130862.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130863.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130864.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130865.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130866.tif
/home/philipp/Code/python/edin_prediction/foto/pred_130867.tif
/home/philipp/Code/python/edin_prediction/foto/pred_131076.tif
/home/philipp/Code/python/edin_prediction/foto/pred_131077.tif
/home/philipp/Code/python/edin_prediction/foto/pred_131078.tif
/home/philipp/Code/python/edin_prediction/foto/pred_131

/home/philipp/Code/python/edin_prediction/foto/pred_133701.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133702.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133703.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133704.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133932.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133933.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133934.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133935.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133936.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133937.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133938.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133939.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133940.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133941.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133942.tif
/home/philipp/Code/python/edin_prediction/foto/pred_133

/home/philipp/Code/python/edin_prediction/foto/pred_136871.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136872.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136873.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136874.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136875.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136876.tif
/home/philipp/Code/python/edin_prediction/foto/pred_136877.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137080.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137081.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137082.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137083.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137084.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137085.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137086.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137087.tif
/home/philipp/Code/python/edin_prediction/foto/pred_137

/home/philipp/Code/python/edin_prediction/foto/pred_139705.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139706.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139707.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139708.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139709.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139710.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139711.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139712.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139713.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139714.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139715.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139716.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139717.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139718.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139719.tif
/home/philipp/Code/python/edin_prediction/foto/pred_139

...

/home/philipp/Code/python/edin_prediction/foto/pred_204357.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204358.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204359.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204360.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204361.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204362.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204363.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204364.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204365.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204366.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204367.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204368.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204369.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204370.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204371.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204

/home/philipp/Code/python/edin_prediction/foto/pred_204757.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204758.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204759.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204760.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204761.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204762.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204763.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204764.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204765.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204766.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204767.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204768.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204769.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204770.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204771.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204

/home/philipp/Code/python/edin_prediction/foto/pred_205419.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205420.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205421.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205422.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205423.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205424.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205425.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205426.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205427.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205428.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205534.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205535.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205536.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205537.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205538.tif
/home/philipp/Code/python/edin_prediction/foto/pred_205

/home/philipp/Code/python/edin_prediction/foto/pred_206967.tif
/home/philipp/Code/python/edin_prediction/foto/pred_206968.tif
/home/philipp/Code/python/edin_prediction/foto/pred_206969.tif
/home/philipp/Code/python/edin_prediction/foto/pred_206970.tif
/home/philipp/Code/python/edin_prediction/foto/pred_206971.tif
/home/philipp/Code/python/edin_prediction/foto/pred_206972.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207100.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207101.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207102.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207103.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207104.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207105.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207106.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207107.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207108.tif
/home/philipp/Code/python/edin_prediction/foto/pred_207

/home/philipp/Code/python/edin_prediction/foto/pred_208273.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208274.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208348.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208349.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208350.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208351.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208352.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208353.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208354.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208355.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208356.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208357.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208358.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208359.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208360.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208

In [7]:
dys = np.stack(d_y)

In [8]:
dys.shape

(10640, 512, 512, 1)

In [9]:
np.save('cm_dys.npy', dys)

In [10]:
d_y = []
#d_y_hat = []

for f in input_files:
    #d_y.append(tif2array("{}{}".format(dir_gt, f[f.rfind('_'):]))[1:-2, 1:-2])
    d_y.append(tif2array(f))
    print(f)

/home/philipp/Code/python/edin_prediction/foto/pred_121889.tif
/home/philipp/Code/python/edin_prediction/foto/pred_121890.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122277.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122278.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122279.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122280.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122647.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122648.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122649.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122650.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122651.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122652.tif
/home/philipp/Code/python/edin_prediction/foto/pred_122653.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123009.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123010.tif
/home/philipp/Code/python/edin_prediction/foto/pred_123

/home/philipp/Code/python/edin_prediction/foto/pred_134880.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134881.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134882.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134883.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134884.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134885.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134886.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134887.tif
/home/philipp/Code/python/edin_prediction/foto/pred_134888.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135111.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135112.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135113.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135114.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135115.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135116.tif
/home/philipp/Code/python/edin_prediction/foto/pred_135

/home/philipp/Code/python/edin_prediction/foto/pred_143473.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143474.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143475.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143476.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143477.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143478.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143479.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143480.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143481.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143482.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143645.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143646.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143647.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143648.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143649.tif
/home/philipp/Code/python/edin_prediction/foto/pred_143

/home/philipp/Code/python/edin_prediction/foto/pred_147816.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147817.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147818.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147819.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147820.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147821.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147822.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147823.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147824.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147825.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147826.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147827.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147828.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147829.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147830.tif
/home/philipp/Code/python/edin_prediction/foto/pred_147

/home/philipp/Code/python/edin_prediction/foto/pred_151373.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151374.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151375.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151376.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151377.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151378.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151379.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151380.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151381.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151382.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151564.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151565.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151566.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151567.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151568.tif
/home/philipp/Code/python/edin_prediction/foto/pred_151

...

/home/philipp/Code/python/edin_prediction/foto/pred_200628.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200629.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200630.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200631.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200632.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200633.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200634.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200635.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200636.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200637.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200638.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200639.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200640.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200641.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200642.tif
/home/philipp/Code/python/edin_prediction/foto/pred_200

/home/philipp/Code/python/edin_prediction/foto/pred_202447.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202448.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202449.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202450.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202451.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202452.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202453.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202454.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202455.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202456.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202457.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202458.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202459.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202460.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202461.tif
/home/philipp/Code/python/edin_prediction/foto/pred_202

/home/philipp/Code/python/edin_prediction/foto/pred_203533.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203534.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203535.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203536.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203537.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203538.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203539.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203540.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203541.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203542.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203543.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203544.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203545.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203546.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203547.tif
/home/philipp/Code/python/edin_prediction/foto/pred_203

/home/philipp/Code/python/edin_prediction/foto/pred_204775.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204776.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204777.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204778.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204869.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204870.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204871.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204872.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204873.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204874.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204875.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204876.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204877.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204878.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204879.tif
/home/philipp/Code/python/edin_prediction/foto/pred_204

/home/philipp/Code/python/edin_prediction/foto/pred_208269.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208270.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208271.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208272.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208273.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208274.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208348.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208349.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208350.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208351.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208352.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208353.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208354.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208355.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208356.tif
/home/philipp/Code/python/edin_prediction/foto/pred_208

In [11]:
dyhats = np.stack(d_y)

In [12]:
np.save('cm_dyhats.npy', dyhats)

In [3]:
y = np.load('cm_dys.npy')

In [4]:
y_hat = np.load('cm_dyhats.npy')

In [4]:
print(y.flatten().shape)
print(y_hat.flatten().shape)

(2789212160,)
(2789212160,)


In [ ]:
stacked = np.stack([y.flatten(), y_hat.flatten()], axis=1)

In [65]:
cmt = np.zeros((5,5))

In [66]:
for p in stacked:
    tl, pl = p.tolist()
    cmt[tl, pl] = cmt[tl, pl] + 1

In [67]:
cmt

array([[     0.,      0.,  19333.,      0.,      0.],
       [     0.,      0.,      0.,      0.,      0.],
       [     0.,      0., 242811.,      0.,      0.],
       [     0.,      0.,      0.,      0.,      0.],
       [     0.,      0.,      0.,      0.,      0.]])

In [21]:
input_files[0][input_files[0].rfind('_'):]

'_121889.tif'

In [22]:
"/media/philipp/DATA/2018_tamsweg/ground_truth_2/tile_ground_truth{}".format(input_files[0][input_files[0].rfind('_'):])

'/media/philipp/DATA/2018_tamsweg/ground_truth_2/tile_ground_truth_121889.tif'